# Demo of functions oneband\_fig, composite\_fig & make\_contact_sheet from*./swiss_utils/data_cube_utilities/sdc_advutils.py*

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy__

Plot onscreen or create a png with a given *xarray.DataArray* (`oneband\_fig`) or *xarray.Dataset* (`composite\_fig`).

These functions 

__oneband_fig__:
* data: one time xarray.DataArray.
* leg: colormap to be applied (either standard (https://matplotlib.org/examples/color/colormaps_reference.html) or custom)
* title: prefix of the figure title
* scalebar_color (OPTIONAL): scalebar color (https://matplotlib.org/examples/color/named_colors.html)
* v_min (OPTIONAL, default minimum value): minimum value to display.
* v_max (OPTIONAL, default maximum value): maximum value to display.
* fig_name (OPTIONAL): file name (including extension) to save the figure (show only if not added to input).
* max_size (OPTIONAL, default 16): maximum size of the figure (either horizontal or vertical).


__composite_fig__:
* data: one time xarray.Dataset containing the three bands mentionned in bands.
* bands: bands to be used in the composite (RGB order).
* title: prefix of the figure title.
* scalebar_color (OPTIONAL): scalebar color (https://matplotlib.org/examples/color/named_colors.html)
* fig_name (OPTIONAL): file name (including extension) to save the figure (show only if not added to input).      
* max_size (OPTIONAL, default 16): maximum size of the figure (either horizontal or vertical).
* hist_str: (OPTIONAL): histogram stretch type (['contr','eq','ad_eq']). Cannot be used with v_min, v_max options.
* v_min (OPTIONAL, default minimum value): minimum value to display. Cannot be used with hist_str option.
* v_max (OPTIONAL, default maximum value): maximum value to display. Cannot be used with hist_str option.

__make_contact_sheet__:
* fnames: a list of names of the image files
* ncols, nrows: number of columns OR rows in the contact sheet
* photow, photoh: width OR eight of the photo thumbs in pixels
* by (OPTIONAL): images displayed by row (default) or columns
* title (OPTIONAL): optional title
* font (OPTIONAL): title font (default LiberationSans bold)
* size (OPTIONAL): title font size (default 14)
* fig_name (OPTIONAL): file name (including extension) to save the figure (show only if not added to input).

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `composite_fig?` or by selecting the function and pressing `Shift-Tab`.

In this demo Jupyter script, the user can either use the in-script function (below) or import it from ./swiss_utils/data_cube_utilities/sdc_advutils.py.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

import os
import glob

import matplotlib.pyplot as plt

from datetime import datetime
from matplotlib import colors
from IPython.display import display, HTML
from IPython.display import Image as IPImage # to avoid conflict with Image function from PIL module

import datacube
dc = datacube.Datacube()

from utils.data_cube_utilities.dc_mosaic import create_hdmedians_multiple_band_mosaic

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_advutils import oneband_fig, composite_fig, natural_sort, make_contact_sheet

The next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `# %load config_cell.txt`.

**Make sure you have at least the measurements ['blue', 'green', 'nir', 'red', 'swir2'] plus the quality band as you will need them all to have a smooth demo!**

In [ ]:
%load config_cell.txt

In [ ]:
draw_map([min_lat, max_lat], [min_lon, max_lon], draw = False)[0]

# composite_fig

In [ ]:
# Create a mosaic
dataset_in, clean_mask = load_multi_clean(dc = dc, products = product, time = [start_date, end_date],
                                          lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                          measurements = measurements)
mosaic = create_hdmedians_multiple_band_mosaic(dataset_in, clean_mask, operation='medoid')
del dataset_in
del clean_mask

In [ ]:
# composite_fig function is recommended as a replacement of write_png_from_xr function from default utils module,
# but let's use it to highlight differences
from utils.data_cube_utilities.dc_utilities import write_png_from_xr

# constrain the diplay range between 0 and 10000 values
write_png_from_xr(png_path = 'write_png_from_xr_example.png',
                  dataset = mosaic,
                  bands = ["red", "green", "blue"],
                  scale = [(200,1500),(200,1500),(200,1500)])

display(HTML("""<a href="write_png_from_xr_example.png" target="_blank" >View and download saved image</a>"""))

In [ ]:
# using composite_fig function without histogram stretch
# uncomment fig_name line to save the figure
composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Demo composite',
#               fig_name = 'demo_composite.png',
              scalebar_color = 'white',
              max_size = 10)

In [ ]:
# with histogram stretch
# uncomment fig_name line to save the figure
composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Demo composite',
#               fig_name = 'demo_composite.png',
              scalebar_color = 'white',
              max_size = 10,
              hist_str = 'contr')

In [ ]:
# you can also also manualy set minimum and maximum values to display
# to do so first plot histogram distribution
kwargs = dict(bins = 50, alpha = 0.3)

mosaic.red.plot.hist(color='red', **kwargs)
mosaic.green.plot.hist(color='green', **kwargs, stacked = True)
mosaic.blue.plot.hist(color='blue', **kwargs, stacked = True)
plt.xlabel('Value')

In [ ]:
# then manually constrain the diplay range using v_min and v_max
# uncomment fig_name line to save the figure
composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Demo composite',
#               fig_name = 'demo_composite.png',
              scalebar_color = 'white',
              v_min = 200,
              v_max = 1500,
              max_size = 10)

In [ ]:
# In the case you would like to save the figure
# simply define the fig_name argument

composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Demo composite',
              fig_name = 'demo_composite.png',
              scalebar_color = 'white',
              v_min = 200,
              v_max = 1500,
              max_size = 10)

In [ ]:
# When saving the figure, only a link is provided
# you can display the figure using the Image function from IPython module
IPImage('demo_composite.png')

# oneband_fig

In [ ]:
ndvi = (mosaic.nir - mosaic.red) / (mosaic.nir + mosaic.red)

In [ ]:
# without fixed color scale (v_min and vmax)
# uncomment fig_name line to save the figure
oneband_fig(ndvi,
            leg = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue',
                                                                    'lightgreen','darkgreen'], N=256),
            title = 'test ndvi scalebar',
#               fig_name = 'test_ndvi_scalebar.png',
            scalebar_color = 'black',
            max_size = 10)

In [ ]:
# with a fixed color scale using v_min and v_max arguments
# and a gold scalebar
# uncomment fig_name line to save the figure
oneband_fig(ndvi,
            leg = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue',
                                                                    'lightgreen','darkgreen'], N=256),
            title = 'test ndvi scalebar',
            fig_name = 'test_ndvi_scalebar.png',
            scalebar_color = 'gold',
            max_size = 10,
            v_min = -1,
            v_max = 1)

# make_contact_sheet

In [ ]:
# Let's create a few normalized index figures first
ndwi = (mosaic.green - mosaic.nir) / (mosaic.green + mosaic.nir)
ndbi = (mosaic.swir2 - mosaic.nir) / (mosaic.swir2 + mosaic.nir)

oneband_fig(ndvi,
            leg = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue',
                                                                    'lightgreen','darkgreen'], N=256),
            title = 'test ndvi',
            fig_name = 'fig_1.png',
            max_size = 10,
            v_min = -1,
            v_max = 1)
oneband_fig(ndwi,
            leg = colors.LinearSegmentedColormap.from_list('ndwi', ['darkgreen', 'lightgreen',
                                                                    'lightblue', 'blue', 'darkblue'], N=256),
            title = 'test ndwi',
            fig_name = 'fig_2.png',
            max_size = 10,
            v_min = -1,
            v_max = 1)
oneband_fig(ndbi,
            leg = colors.LinearSegmentedColormap.from_list('ndbi', ['beige', 'black'], N=256),
            title = 'test ndbi',
            fig_name = 'fig_3.png',
            max_size = 10)

In [ ]:
# Let's create a contact sheet by rows (default)
make_contact_sheet(natural_sort(glob.glob('fig*.png')), ncols = 2, photow = 500,
                   title = 'All ND*Is ata once')

In [ ]:
# and the same by columns
make_contact_sheet(natural_sort(glob.glob('fig*.png')), ncols = 2, photow = 500,
                   title = 'All ND*Is ata once',
                   by = 'col',)

In [ ]:
# as previous functions you can save the figure simply by adding the fig_name argument
make_contact_sheet(natural_sort(glob.glob('fig*.png')), ncols = 2, photow = 500,
                   title = 'All ND*Is ata once',
                   fig_name = 'all_ndis.png')